# Dependencies

In [2]:
# standard libraries
#
import os

# mathematical and scientific
#
import numpy as np
# import sklearn as skl
# from sklearn import cluster

# geoprocessing libraries
#
# import shapely as shp
import rasterio as rio
from rasterio import features
from rasterio.plot import show_hist as hist
import fiona

# Data

In [3]:
work_dir = "./data/"
os.makedirs(work_dir, exist_ok=True)
image = f"{work_dir}LWIR_QuickMosaic_16-bit_9327.tiff"

# Classify

In [4]:
with rio.open(image) as src:
    band1 = src.read(1)
    profile = src.profile

    # for thresh in range(30000, 36000, 1000):
    # for thresh in range(33000, 34000, 100):
    # for thresh in range(33300, 33400, 10):
    for thresh in range(33330, 33340, 1):

        with rio.open(f"{work_dir}class{thresh}.tif", 'w', **profile) as dst:
            imgarr = np.where(band1 < thresh, 0, 1)
            # centers, indices = cluster.kmeans_plusplus(imgarr, n_clusters=8, random_state=0)
            # print(len(centers))
            # print(f"{thresh}: {indices}")
            dst.write(imgarr.astype(rio.uint8), 1)
    
    # hist(src, bins=256, )

# this was found manually via trial and error above for given test tiff
fire_treshold = 33332
# TODO: find this automatically!

# Aggregate Polygons

In [6]:
best = f"{work_dir}class{fire_treshold}.tif"
with rio.open(best) as src:
    band1 = src.read(1)
    profile = src.profile
    print(profile)

    #imgarr = np.where(band1 < thresh, 0, 1)
    #imgarr_uint8 = imgarr.astype(rio.uint8)

    # Convert the classified raster to polygons
    shapes = features.shapes(band1, transform=profile['transform'])

with fiona.open(f"{work_dir}class{fire_treshold}.shp", 'w', 'ESRI Shapefile',
                crs=fiona.crs.from_epsg(4326), # this is needed to create prj file
                # crs=fiona.crs.from_epsg(26910), # NAD83 UTM zone 10N THIS FAILS!
                schema={'geometry': 'Polygon', 'properties': {}}) as dst:
    for shape in shapes:
        value = shape[1]
        if value > 0:
            feature = {'geometry': shape[0], 'properties': {}}
            dst.write(feature)


{'driver': 'GTiff', 'dtype': 'uint16', 'nodata': None, 'width': 5184, 'height': 12800, 'count': 1, 'crs': CRS.from_epsg(4326), 'transform': Affine(1.671045524691221e-05, 0.0, -122.446287,
       0.0, -1.2611718749999667e-05, 41.553101000000005), 'blockxsize': 256, 'blockysize': 256, 'tiled': True, 'compress': 'deflate', 'interleave': 'band'}
